\
            # 41. MySQL：基础与最佳实践（MySQL Basics & Practices）

            覆盖关系型数据库的核心概念、Python 连接与参数化查询、事务与索引等。
示例默认提供可运行的 `sqlite3` 降级版本，并说明如何切到 MySQL。

            > 约定：Python 3.8；示例尽量只用标准库；代码块可直接运行（第三方依赖会做可选降级）。


## 前置知识

- SQL 基础（SELECT/INSERT/UPDATE/DELETE）
- 异常与 with
- 字典/列表


## 知识点地图

- 1. 表设计：类型、约束与索引（够用即可）
- 2. Python 访问：连接、游标、参数化查询
- 3. 事务：一致性与失败处理
- 4. 索引与查询：看懂 EXPLAIN（概念）
- 5. 连接池与迁移（工程化）
- 6. 切到 MySQL：驱动与环境变量（可选）


## 自检清单（学完打勾）

- [ ] 能解释主键/唯一约束/外键的目的
- [ ] 会写参数化查询，避免 SQL 注入
- [ ] 理解事务、提交/回滚与隔离级别（概念）
- [ ] 知道索引的作用与基本代价
- [ ] 知道连接池与迁移在工程里的位置（概念）


In [ ]:
\
from pathlib import Path

ART = Path('_nb_artifacts')
ART.mkdir(exist_ok=True)
print('artifacts dir:', ART.resolve())


## 知识点 1：表设计：类型、约束与索引（够用即可）

核心目标：保证数据正确（约束）+ 查询快（索引）+ 可演进（迁移）。

- 约束：PRIMARY KEY、UNIQUE、NOT NULL、FOREIGN KEY（可选）
- 索引：加速查询，但会让写入更慢、占更多空间
- 面试常问：为什么要主键？为什么要唯一约束？索引为什么会让写入变慢？


## 知识点 2：Python 访问：连接、游标、参数化查询

- 参数化查询使用占位符（驱动会处理转义），不要拼接 SQL 字符串。
- `commit()` 提交；异常时 `rollback()`。
- 连接/游标要正确关闭：优先用 `with` 或 try/finally。

下面用 sqlite3 演示完整流程（可直接运行）。


In [ ]:
import sqlite3


db = sqlite3.connect(':memory:')
db.execute('create table user(id integer primary key, name text not null, age int not null)')
db.execute('insert into user(name, age) values(?, ?)', ('Alice', 18))
db.execute('insert into user(name, age) values(?, ?)', ('Bob', 20))
db.commit()

rows = db.execute('select id, name, age from user where age >= ?', (19,)).fetchall()
print(rows)


## 知识点 3：事务：一致性与失败处理

事务（Transaction）让一组操作要么全部成功，要么全部失败回滚。

常见失败处理套路：
- begin/commit
- except: rollback + 记录日志 + 返回错误

隔离级别影响并发读写行为（脏读/不可重复读/幻读）：工程里要结合业务与 MySQL 配置选。


In [ ]:
import sqlite3


db = sqlite3.connect(':memory:')
db.execute('create table account(id integer primary key, name text, balance int)')
db.executemany('insert into account(name, balance) values(?, ?)', [('A', 100), ('B', 100)])

def transfer(frm, to, amount):
    try:
        db.execute('begin')
        db.execute('update account set balance = balance - ? where name = ?', (amount, frm))
        # 模拟故障
        raise RuntimeError('something bad')
        db.execute('update account set balance = balance + ? where name = ?', (amount, to))
        db.commit()
    except Exception:
        db.rollback()


transfer('A', 'B', 30)
print(db.execute('select name, balance from account order by name').fetchall())


## 知识点 4：索引与查询：看懂 EXPLAIN（概念）

- 索引通常是 B+Tree（MySQL/InnoDB 常见），加速“按条件查找”。
- 组合索引要注意“最左前缀”。
- 慢查询排查：慢查询日志 -> EXPLAIN -> 缺索引/扫描行数过多/回表。


## 知识点 5：连接池与迁移（工程化）

- 连接池：避免频繁建连；控制最大连接数；通常由框架/库提供（SQLAlchemy、DBUtils 等）。
- 迁移：用工具管理 schema 版本（Alembic/Flyway/Liquibase），不要手工改线上表。
- 安全：最小权限账号、参数化查询、审计、备份与恢复演练。


## 知识点 6：切到 MySQL：驱动与环境变量（可选）

如果你本机有 MySQL：
1) 安装驱动（任选其一）
   - `pip install mysql-connector-python`
   - `pip install pymysql`
2) 配置环境变量：
   - `MYSQL_HOST`, `MYSQL_PORT`, `MYSQL_USER`, `MYSQL_PASSWORD`, `MYSQL_DATABASE`

下面给出“优先 MySQL，失败就用 sqlite3”的连接模板。
（注意：不同驱动占位符可能不同；示例里对常见两类做了分支。）


In [ ]:
import os
import sqlite3


def connect_db():
    host = os.getenv('MYSQL_HOST')
    user = os.getenv('MYSQL_USER')
    password = os.getenv('MYSQL_PASSWORD')
    database = os.getenv('MYSQL_DATABASE')
    port = int(os.getenv('MYSQL_PORT', '3306'))

    if host and user and password and database:
        try:
            import mysql.connector  # type: ignore

            conn = mysql.connector.connect(host=host, port=port, user=user, password=password, database=database)
            return 'mysql-connector', conn
        except Exception:
            pass

        try:
            import pymysql  # type: ignore

            conn = pymysql.connect(host=host, port=port, user=user, password=password, database=database)
            return 'pymysql', conn
        except Exception:
            pass

    return 'sqlite3', sqlite3.connect(':memory:')


driver, conn = connect_db()
print('driver:', driver)
conn.close()


## 常见坑

- 拼接 SQL 字符串：SQL 注入风险
- 忘记 commit/rollback：数据不一致或锁不释放
- 滥建索引：写入变慢、维护成本高
- 深分页用 LIMIT/OFFSET：越往后越慢（可用 seek pagination）


## 综合小案例：最小 Todo CRUD（sqlite3 可跑，MySQL 可迁移）

- 创建 todo 表（id, title, done, created_at）
- 实现 create/list/mark_done/delete
- 演示参数化与事务


In [ ]:
import sqlite3
import time


db = sqlite3.connect(':memory:')
db.execute('create table todo(id integer primary key, title text not null, done int not null, created_at int not null)')

def create(title: str):
    ts = int(time.time())
    db.execute('insert into todo(title, done, created_at) values(?, ?, ?)', (title, 0, ts))
    db.commit()


def list_all():
    return db.execute('select id, title, done, created_at from todo order by id').fetchall()


def mark_done(todo_id: int):
    db.execute('update todo set done = 1 where id = ?', (todo_id,))
    db.commit()


def delete(todo_id: int):
    db.execute('delete from todo where id = ?', (todo_id,))
    db.commit()


create('learn sql')
create('learn transactions')
print(list_all())
mark_done(1)
delete(2)
print(list_all())


## 自测题（不写代码也能回答）

- 为什么必须用参数化查询？它如何避免 SQL 注入？
- 事务解决了什么问题？commit/rollback 分别做什么？
- 为什么索引会让写入变慢？


## 练习题（建议写代码）

- 把小案例改成“软删除”（deleted_at），并确保查询默认过滤。
- 实现 seek pagination：按 (created_at, id) 向后翻页。
- 把 sqlite3 版本迁移到 MySQL（任选驱动），并用环境变量配置连接。
